In [26]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt # for plotting the data 
import seaborn as sns # Advanced data plotting on top of matplotlib

import scipy.sparse as sp

In [27]:
#считывание данных


users = pd.read_csv('../input/tumenrecsys/train/users.csv', sep=';', index_col=None, dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})
items = pd.read_csv('../input/tumenrecsys/train/items.csv', sep=';', index_col=None, dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str})
train_transactions = pd.read_csv('../input/tumenrecsys/train/train_transactions_extended.csv', sep=';', index_col=None, dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str})

In [28]:
#train_transactions.sys_numb.unique().shape

In [29]:
users.head(10)

,chb,age,gender,chit_type
0,300001020830,21,female,нет данных
1,300001113642,36,female,нет данных
2,300001148466,46,female,нет данных
3,300001117011,22,female,нет данных
4,200001038094,24,female,echb
5,200001120504,22,female,нет данных
6,300000849123,27,male,нет данных
7,300001002599,21,female,нет данных
8,200000931895,28,female,нет данных
9,300000806816,23,female,normal


In [30]:
items.head(10)

,sys_numb,title,author,izd,year_izd,bbk
0,RSL01008600016,Судебное следствие в уголовном процессе России...,"Машовец, Асия Океановна",Юрлитинформ,2016,"Х629.374,0"
1,RSL01004304880,Уральское казачество и его роль в системе росс...,"Дубовиков, Александр Маратович",none,2006,none
2,RSL07000461043,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует
3,RSL07000433335,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует
4,RSL01002419013,"""Я пишу как эхо другого.."" : Очерки интеллекту...","Щедрина, Татьяна Геннадьевна",Прогресс-Традиция,2004,"Ю3(2)6-69Шпет Г.Г.-3,0"
5,RSL01004307424,Использование дробных физических нагрузок в ст...,"Статун, Алла Николаевна",none,2007,none
6,RSL07000467555,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует
7,RSL01009483558,Очерк психологии полиморфной индивидуальности ...,"Белоус, Валерий Владимирович",none,2016,none
8,RSL01004292734,"ФАКТОРЫ, ОПРЕДЕЛЯЮЩИЕ ПРИВЕРЖЕННОСТЬ К ЛЕЧЕНИЮ...","Бирюкова, Любовь Алексеевна",none,2009,none
9,RSL07000458490,отсутствует,отсутствует,отсутствует,отсутствует,отсутствует


In [31]:
items.shape

(354355, 6)

In [32]:
#items.drop(items[items["title"] == "отсутствует"].index, inplace=True)

In [33]:
#т.к далее придётся работать с матрицами созданим словари, которые точно отображают индексы в chb/sys_numb и обратно

mapping_chb2index = {chb_number: index for index, chb_number in enumerate(users.chb.values)}
mapping_index2chb = {index: chb_number for index, chb_number in enumerate(users.chb.values)}

mapping_sys_numb2index = {sys_numb: index for index, sys_numb in enumerate(items.sys_numb.values)}
mapping_index2sys_numb= {index: sys_numb for index, sys_numb in enumerate(items.sys_numb.values)}


# EDA

In [34]:
nodata_books = set(items[items["title"] == "отсутствует"]["sys_numb"].values)

In [35]:
train_transactions.duplicated().sum()

0

In [36]:
for col in items.columns:
    print(col, len(items[col].unique()))

sys_numb 354355
title 319821
author 172622
izd 36909
year_izd 2506
bbk 130780


In [37]:
train_transactions.drop_duplicates(subset=["chb","sys_numb"], keep='first', inplace=True, ignore_index=False)

In [38]:
len(set(train_transactions.sys_numb).intersection(nodata_books))  

14796

In [39]:
print(train_transactions.groupby("chb")["sys_numb"].count().mean())
train_transactions.groupby("chb")["sys_numb"].count().sort_values()

13.669312958873038


chb
300000884658       2
100001111620       2
300000838729       2
300000838696       2
300000838230       2
                ... 
300000933517    1288
300001058749    1297
300000758346    1655
200000789223    3914
200000872413    6671
Name: sys_numb, Length: 16753, dtype: int64

In [40]:
# Популярное
most_popular_index = train_transactions.groupby("sys_numb")["chb"].count().sort_values().tail(50).index 
for k in most_popular_index:
    print(items[items["sys_numb"] == k]["title"].values[0])

Церковная археология Москвы : храмы и приходы Ивановской горки и Кулишек
Всеобщая история интерьера 
Москва в кольце Садовых : путеводитель
История русского искусства : Русское искусство XX века
Архитектура советского авангарда : в 2 книгах / С. О. Хан-Магомедов : Социальные проблемы
С Рождеством Христовым! Изоматериал : Открытка
С Рождеством Христовым Изоматериал : Открытка
Зодчие Санкт-Петербурга, XVIII в. : Архитекторы барокко. Ранний классицизм. Строгий классицизм : Очерки
Мебель в русском интерьере конца XIX - начала XX веков
И по плодам узнается древо : рус. иконопись XV-XX веков из собр. Виктора Бондаренко
Русский стиль : Поиски выражения нац. самобытности. Народность и национальность. Традиции древнерус. и нар. искусства в рус. искусстве XVIII-нач. XX в.
С Рождеством Христовым Изоматериал : Открытка
Русские живописцы XVIII-XIX века : биографический словарь
С Рождеством Христовым! Изоматериал : Открытка
На рубеже веков… : искусство эпохи модерна
Теория литературы : Учеб. для сту

**Вякие нужные словарики**

In [41]:

mapping_chb2index = {chb_number: index for index, chb_number in enumerate(users.chb.values)}
mapping_index2chb = {index: chb_number for index, chb_number in enumerate(users.chb.values)}

mapping_sys_numb2index = {sys_numb: index for index, sys_numb in enumerate(items.sys_numb.values)}
mapping_index2sys_numb= {index: sys_numb for index, sys_numb in enumerate(items.sys_numb.values)}

In [42]:
train_transactions["chb_i"] = train_transactions["chb"].apply(lambda a: mapping_chb2index[a])
train_transactions["sys_num_i"] = train_transactions["sys_numb"].apply(lambda a: mapping_sys_numb2index[a])

In [43]:
most_popular = [mapping_sys_numb2index[item] for item in most_popular_index[:20]]
#most_popular = most_popular_index[15:40].values
most_popular = most_popular_index[-25:]
most_popular

Index(['RSL01003033735', 'RSL01000647092', 'RSL01004116615', 'RSL01000669023',
       'RSL01000619379', 'RSL01009823884', 'RSL01002789241', 'RSL01006826042',
       'RSL01004353863', 'RSL01009824080', 'RSL01009824102', 'RSL01009817125',
       'RSL01004271448', 'RSL01002884697', 'RSL01009815656', 'RSL01000741775',
       'RSL01009819690', 'RSL01009817128', 'RSL01003187450', 'RSL01002890388',
       'RSL01009819689', 'RSL01009824143', 'RSL01009819687', 'RSL01009824134',
       'RSL01009819710'],
      dtype='object', name='sys_numb')

In [44]:
mapping_sys_numb2index = {row["sys_numb"]:row["author"] for _, row in items.iterrows()}
#mapping_sys_numb2index

**Подгрузка данных из первой части решения**

In [45]:
# Решение полученные с помощью BERT модели !!!

nlp_recs = pd.read_csv("../input/fork-of-tumen-recsys/solution_02.csv", sep=";")
nlp_recs.head(30)
nlp_recs[nlp_recs.chb == 300001113642]

,chb,sys_numb
20,300001113642,RSL01002498412
21,300001113642,RSL01003157807
22,300001113642,RSL01003743647
23,300001113642,RSL01003675063
24,300001113642,RSL01002566448
25,300001113642,RSL01003669646
26,300001113642,RSL01004202363
27,300001113642,RSL01003600832
28,300001113642,RSL01003688594
29,300001113642,RSL01003553160


# Конвеер

In [46]:
# Отладочная версия конвеера с колокольчиками и свистками

all_rec = []

#embeddings = title_vecs_w2v

# go through all customers from test list
for customer in users.chb.values[0:7]:
    
    
    # go through all purchases of customers
    customer_pred = []
    
    print(customer)
    print("*"*40, "Ранее запрошено:", "*"*40)
    
    for k in train_transactions[train_transactions.chb == customer].sys_numb.values:
        print(items[items["sys_numb"] == k]["title"].values[0], items[items["sys_numb"] == k]["author"].values[0])
    
    print("*"*40)
    
    num = len(train_transactions[train_transactions.chb == customer].sys_numb)
    
    
    #samp_n = max(20%num, 1)
    
    item_author = []
    
    for sys_numb in train_transactions[train_transactions.chb == customer].sys_numb:
        
        if sys_numb in nodata_books: continue

        
        author = mapping_sys_numb2index[sys_numb]
        
        if author in item_author:
            continue
        else:
            item_author.append(author)

        
        print("Поиск Автора --->", author)
        
        author_books = items[items["author"] == author]
        
        #print(author_books.head(3))
        
        pred = list(author_books["sys_numb"].values[:5])
        
        if sys_numb in pred:
            pred.remove(sys_numb)
         
        customer_pred += pred[:2]
        
        if len(np.unique(customer_pred)) > 20:
            break
        
    customer_pred = list(np.unique(customer_pred)[:20])
    
    if len(customer_pred) < 20:
        #print(customer, nlp_recs[nlp_recs['chb'] == int(customer)])
        print(len(customer_pred))
        
        for a in nlp_recs[nlp_recs.chb == int(customer)]['sys_numb'].values[:(20-len(customer_pred))]:
            #print("%%%%", a)
            customer_pred.append(a)
            
        #print(customer_pred)
        
        #for a in most_popular[:(20-len(customer_pred))]:
        #    customer_pred.append(a)
        #print(customer_pred)

    print(len(customer_pred))
    
    print("*"*40, "Рекомендовано:", "*"*40)
    for book in customer_pred:
        #book = mapping_index2sys_numb[k]
        all_rec.append([customer, book])
        print(items[items["sys_numb"] == book]["title"].values[0])
    print("--"*20)    
    
    #full_pred_list.append(np.unique(customer_pred))
    print()
    print()
    print()

300001020830
**************************************** Ранее запрошено: ****************************************
Европейские стандарты справедливого судебного разбирательства по уголовным делам : монография Васяев, Александр Александрович
Уголовная политика Европейского союза в сфере противодействия коррупции : монография Краснова, Кристина Александровна
отсутствует отсутствует
отсутствует отсутствует
Проблемы реализации принципов криминализации и положений законодательной техники применительно к нормам главы 22 УК РФ : диссертация .. кандидата юридических наук : 12.00.08 Лозинский, Игорь Владиславович
Основы законодательной техники в уголовном праве России : теория и законодательная практика : диссертация .. доктора юридических наук : 12.00.08 Панько, Кирилл Константинович
Криминализация (декриминализация) в уголовном праве России : диссертация .. кандидата юридических наук : 12.00.08 Радошнова, Наталия Валерьевна
Законодательная техника в уголовном праве : автореферат дис.. кандидата 

In [47]:
import time

In [48]:
%%time

all_rec = []
ct = time.time()


# go through all customers from test list
for ii, customer in enumerate(users.chb.values):
    
    if ii % 100 == 0:
        print(ii, "/", 16753)# all_len)
    
        print(time.time() - ct)
        ct = time.time()
    
    
    # go through all purchases of customers
    customer_pred = []
    
    item_author = []
    
    for sys_numb in train_transactions[train_transactions.chb == customer].sys_numb:
        
        # Пропускаем книги без авторства
        if sys_numb in nodata_books: continue

        
        author = mapping_sys_numb2index[sys_numb]
        
        # Если автор попададся продолжим
        if author in item_author:
            continue
        else:
            item_author.append(author)
        
        # Подборка книг автора
        author_books = items[items["author"] == author]
        
        pred = list(author_books["sys_numb"].values[:5])
        
        if sys_numb in pred:
            pred.remove(sys_numb)
         
        customer_pred += pred[:2]
        
        if len(np.unique(customer_pred)) > 20:
            break
        
    customer_pred = list(np.unique(customer_pred)[:20])
    
    
    # Если авторских книг малодополняем NLP
    if len(customer_pred) < 20:       
        for a in nlp_recs[nlp_recs.chb == int(customer)]['sys_numb'].values[:(20-len(customer_pred))]:
            customer_pred.append(a)

            
    for book in customer_pred:
        all_rec.append([customer, book])


        
        
all_rec[:3]

0 / 16753
0.00044989585876464844
100 / 16753
57.56617498397827
200 / 16753
50.105844259262085
300 / 16753
41.34984993934631
400 / 16753
38.91732835769653
500 / 16753
37.118985652923584
600 / 16753
30.393611431121826
700 / 16753
33.53188371658325
800 / 16753
27.407939434051514
900 / 16753
29.026288509368896
1000 / 16753
32.27306365966797
1100 / 16753
29.13502049446106
1200 / 16753
24.67293405532837
1300 / 16753
20.317418336868286
1400 / 16753
20.965456008911133
1500 / 16753
22.449695110321045
1600 / 16753
28.033241987228394
1700 / 16753
24.009559392929077
1800 / 16753
37.00450086593628
1900 / 16753
50.03057813644409
2000 / 16753
42.15892815589905
2100 / 16753
34.950496435165405
2200 / 16753
33.9963595867157
2300 / 16753
36.3387086391449
2400 / 16753
29.373666763305664
2500 / 16753
33.08627510070801
2600 / 16753
26.61104726791382
2700 / 16753
24.770959854125977
2800 / 16753
25.137286901474
2900 / 16753
23.522294998168945
3000 / 16753
27.29049253463745
3100 / 16753
24.133008003234863
3200

[['300001020830', 'RSL01000196558'],
 ['300001020830', 'RSL01000259987'],
 ['300001020830', 'RSL01000736234']]

In [49]:
# DataFrame для отправки решения с рекомендациями
solution = pd.DataFrame(all_rec, columns=["chb", "sys_numb"])
print(solution.chb.unique().shape)
print(solution.sys_numb.unique().shape)
solution.to_csv("solution_050922_4.csv", index=False, sep=';')

(16753,)
(132471,)


In [50]:
solution

,chb,sys_numb
0,300001020830,RSL01000196558
1,300001020830,RSL01000259987
2,300001020830,RSL01000736234
3,300001020830,RSL01001434460
4,300001020830,RSL01002302503
...,...,...
335055,300001155049,RSL01002623623
335056,300001155049,RSL01002181694
335057,300001155049,RSL01000761005
335058,300001155049,RSL01004073072
